In [120]:
import requests, re

def format_msg(msg): 
    if type(msg) is list:
        return [{"role": "user", "content": str(m)} for m in msg]
    else:
        return [{"role": "user", "content": str(msg)}]

def query_local_llm(msgs, limit=4000) -> tuple[str, float]:
    # Replace with your actual server address and port
    url = "http://0.0.0.0:11434/api/chat/"
    payload = {
        "model": 'codestral',
        "messages" : format_msg(msgs),
        "stream": False,
        "options": {
            "temperature": 1,
            "num_predict": limit,
        }
    }

    response = requests.post(url, json=payload)
    if response.status_code == 200:
        generation_dict = dict(response.json())
        comp_tok = generation_dict.get('eval_count', 0)
        output = generation_dict['message']['content']
        return output, comp_tok 
    else:
        print(f"Error: {response.status_code}, {response.text}")

In [121]:
test_question = "Let's play challenge 24! The goal of the game is to use arithmetic operations (+, -, x, /) to find a combination of the given numbers that equal 24. You must use all of the numbers and you can only use each number once. Here is your problem: (2, 6, 1, 6)."
test_thought = '''2 * 6 + 1 * 6 = 24

The solution to this problem involves multiplying 2 by 6 to get 12, then multiplying 1 by 6 to get 6. Adding these together results in the target number of 24.'''

t, ct = query_local_llm([test_question])

In [122]:
print(t)

2 * 6 + 1 * 6 = 24

The solution to this problem involves multiplying 2 by 6 to get 12, then multiplying 1 by 6 to get 6. Adding these together results in the target number of 24.


In [123]:
class ThoughtNode:
    def __init__(self, thought, goal, parent=None) -> None:
        self.parent = parent
        self.thought = thought
        self.width = 5
        self.children = []
        self.get_score(goal)

    def __str__(self):
        return f'''Node:
    Score: {self.score}
    Thought: {self.thought}
    Children: {self.children}
    Parent: {self.parent}
    '''

    def get_thought_process(self):
        return self.thought if self.parent is None else self.parent.get_thought_process() + f'\n\n{"-"*10}\n' + self.thought

    @staticmethod
    def extract_score(response):
        pattern = r"Score:\s*(\d+(?:\.\d+)?)"
        matches = re.findall(pattern, response, re.MULTILINE)
        if matches:
            return float(matches[0])
        else:
            return 0.5

    def get_score(self, problem):
        prompt = f"Evaluate how promising this thought is for solving the problem '{problem}' on a scale of 0 to 10: '{self.thought}' Make sure your response is formatted as follows 'Score: [value 0-10]'"
        scores = []
        for _ in range(3):
            response, _ = query_local_llm(prompt)
            sc = self.extract_score(response)
            print(sc)
            scores.append(sc)
        self.score = sum(scores)/len(scores)

    def add_child(self, t: str, g:str):
        node = ThoughtNode(t, g, parent=self)
        self.children.append(node)

    def generate_children(self, problem):
        prompt = f'We are attempting to solve this problem "{problem}", and we currently have this partial solution "{self.get_thought_process()}". Given this information please generate the next step to solving the problem.'
        num_nodes = max(0, int(self.width * self.score)//10)

        for _ in range(num_nodes):
            cur_t, _ = query_local_llm(prompt)
            self.add_child(cur_t, problem)
    

In [124]:
test_node = ThoughtNode(test_thought, test_question)

9.0
0.5
8.0


In [125]:
print(test_node.get_thought_process())

2 * 6 + 1 + 6 = 24

So, the combination is: (2 * 6) + 1 + 6


In [126]:
test_node.generate_children(test_question)

6.0
7.0
7.0
9.0
8.0
9.0


In [128]:
print(len(test_node.children))

2


In [127]:

for n in test_node.children:
    print(n.get_thought_process())

2 * 6 + 1 + 6 = 24

So, the combination is: (2 * 6) + 1 + 6

----------
2*6 will give you a result of 12. So, based on your current progress, the equation can be written as 12 + 1 + 6.

Next steps could include:
- Adding or subtracting 1 and 6 to get closer to 24. For example, adding 1 to 12 gives you 13 which is still less than 24. So, you might want to try adding 6 to this result.
- Trying other arithmetic operations with the numbers or rearranging them to form a new combination. However, given the current equation, there doesn't seem to be another immediate solution without breaking the rule of using each number only once.
2 * 6 + 1 + 6 = 24

So, the combination is: (2 * 6) + 1 + 6

----------
2 * 6 + 1 + 6 equals 13, which is not equal to 24. We need to find a new combination that results in 24 using the remaining numbers (2, 6, and 6).

Given this, one possible next step could be to subtract or divide instead of adding:
- Subtraction: Try "2 * 6 - 1 - 6" or "2 * 6 - 1 + 6" to find

In [ ]:
class ThoughtTree:
    def __init__(self, problem, max_depth=3, beam_width=3) -> None:
        self.root = ThoughtNode(problem)
        self.max_depth = max_depth
        self.beam_width = beam_width

    def generate_thoughts(self, node: ThoughtNode) -> ThoughtNode:
        prompt = f"Given the problem or previous thought: '{node.thought}', generate the possible next thought or step that will help achieve the goal."
        for _ in range(self.beam_width):
            response, _ = query_local_llm(prompt)
            thoughts = response.strip()
            node.add_child(thoughts)
        return node

In [ ]:
problem = input('Enter a Query: ')
tree = ThoughtTree(problem)
solution = tree.search()

print("Problem:\n", problem, '\n')
print("\nSolution path:")
for i, thought in enumerate(solution):
    print(f"{i+1}:\n{thought}\n")